In [3]:
%load_ext autoreload
%autoreload 2

import os
import torch
import torchaudio

from audio_utils import print_stats, plot_waveform, plot_specgram, play_audio, plot_spectrogram
from dataloader_audio import AudioDataset

print(f"Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")
# print(f"torchaudio {torchaudio.__version__}")

AUDIO_DIR = 'data/audio'
LABEL_DIR = 'data/label'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using torch 1.9.1 (NVIDIA GeForce RTX 3060 Laptop GPU)


## Dataset

In [4]:
training_data = AudioDataset(AUDIO_DIR, LABEL_DIR)
test_data = AudioDataset(AUDIO_DIR, LABEL_DIR)

# print('First iteration of data set: ', next(iter(training_data)))
# print(len(training_data))
# print(len(test_data))

/home/xunwei/miniconda3/envs/asr/lib/python3.9/site-packages/torchaudio/functional/functional.py:432: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


## DataLoader

In [5]:
train_dataloader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

In [6]:
# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
mfcc = train_features[0][0]
label = train_labels[0]

plot_spectrogram(mfcc)
print(f"Label: {label}")

AttributeError: 'AudioDataset' object has no attribute 'text_process'

## Transform

In [9]:
char_map_str = """
 ' 0
 <SPACE> 1
 a 2
 b 3
 c 4
 d 5
 e 6
 f 7
 g 8
 h 9
 i 10
 j 11
 k 12
 l 13
 m 14
 n 15
 o 16
 p 17
 q 18
 r 19
 s 20
 t 21
 u 22
 v 23
 w 24
 x 25
 y 26
 z 27
 """
char_map_str

"\n ' 0\n <SPACE> 1\n a 2\n b 3\n c 4\n d 5\n e 6\n f 7\n g 8\n h 9\n i 10\n j 11\n k 12\n l 13\n m 14\n n 15\n o 16\n p 17\n q 18\n r 19\n s 20\n t 21\n u 22\n v 23\n w 24\n x 25\n y 26\n z 27\n "

In [10]:
class TextTransform:
    def __init__(self):
        char_map_str = char_map_str
        self.char_map = {}
        self.index_map = {}
        for line in char_map_str.strip().split('\n'):
            ch, index = line.split()
            self.char_map[ch] = int(index)
            self.index_map[int(index)] = ch
        self.index_map[1] = ' '
        
    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('', ' ')

In [13]:
train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=15),
    torchaudio.transforms.TimeMasking(time_mask_param=35)
)

## Model

In [5]:
from torch import nn

In [ ]:
class NeuralNetwork(nn.Module):
    super(NeuralNetwork, self).__init__()
    self.

In [9]:
def label_to_index(word):
    # Return the position of the word in labels
    return torch.tensor(labels.index(word))


def index_to_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return labels[index]


word_start = "yes"
index = label_to_index(word_start)
# word_recovered = index_to_label(index)

print(word_start, "-->", index, "-->", word_recovered)

NameError: name 'labels' is not defined